In [ ]:
# test sample code
#%run -i sample-code.py
malware_classes = ["Agent", "AutoRun", "FraudLoad", "FraudPack", "Hupigon", "Krap",
           "Lipler", "Magania", "None", "Poison", "Swizzor", "Tdss",
           "VB", "Virut", "Zbot"]
val = ""
for cls in malware_classes:
    val += cls + ","

feat_imp = np.loadtxt( "feature_importances.csv", delimiter = ","  )
pct = np.percentile(feat_imp,40)
imp_idx = feat_imp > 0
len(feat_imp[imp_idx])

len( t_train ) /2

val



In [1]:
import sample_code as sc
import numpy as np
import cPickle
reload( sc )
#find valid tags, and save it, just run it once
#sc.find_all_tags(0, 10000, direc="train")
#valid_tags = np.load( "valid_tags.npy" )

#with open("valid_tags.pckl") as f:
#    valid_tags = cPickle.load(f)

# extract features for both training and testing samples, just run once, unless to extract other features
# sc.save_feature_data()

# load data
with open("train_features.pckl") as f:
    X_train, t_train, train_ids = cPickle.load(f)

with open("test_features.pckl") as f:
    X_test, t_test, test_ids = cPickle.load(f)

In [2]:
# use sklearn, RF, different parameters, test in-sample and out-sample results, look at mis-classification and important features
# and matrix of misclassification rates
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from LogisticRegression import LogisticRegression
from GaussianGenerativeModel import GaussianGenerativeModel
from sklearn.svm import SVC
%matplotlib inline

In [ ]:
X_train.shape

In [37]:
# test different algo and get error stats
def eval_results( bmk, pred ):
    err = 0.0
    mat = np.zeros((15,15))
    for idx, val in enumerate(pred):
        act = bmk[idx]
        if act != val:
            err += 1.0
            mat[int(act),int(val)] += 1.0
    err /= len(bmk)
    return err, mat

method = "GG"
method = "LR"
method = "ET"
method = "RF"
method = "SV"

tot = len( t_train )
tr_set = range(0,tot/2)
val_set = range(tot/2,tot)
#val_set,tr_set = tr_set,val_set
prefix = method + "_poly_cont_tr_1hf"
X_train_enh = X_train.copy()
# change to binary
#X_train_enh[ X_train_enh > 0.0 ] = 1.0

# normalize
#shp = X_train_enh.shape
#X_train_enh = X_train_enh / X_train_enh.dot( np.ones( ( shp[1], shp[1] ) ) )

# important features only
feat_imp = np.loadtxt( "feature_importances.csv", delimiter = ","  )
pct = np.percentile(feat_imp,40)
imp_idx = feat_imp > pct
# X_train_enh = X_train_enh[:,imp_idx]

if( method == "RF" ):
    n_est = 30
    n_ft = 10 # 30 or 10 both seem OK     
    min_split = 4 # min
    if True: # for min_split in [1,2,3,4,5,6]:
        regr = RandomForestClassifier(n_estimators = n_est, max_features = n_ft, min_samples_split = min_split )
        regr.fit( X_train_enh[tr_set,:], t_train[tr_set] )
        # get in sample and out of sample errors
        Y_in = regr.predict( X_train_enh[tr_set,:] )
        Y_out = regr.predict( X_train_enh[val_set,:] )
        err_in, mat_in = eval_results( t_train[tr_set], Y_in )
        err_out, mat_out = eval_results( t_train[val_set], Y_out )   
        np.savetxt(prefix+"_mat_in.csv", mat_in, delimiter = ',')
        np.savetxt(prefix+"_mat_out.csv", mat_out, delimiter = ',')
        print prefix, "n_est:", n_est, "max_ft", n_ft, "min_sp_spl", min_split, "in-sp error:", err_in, "out-sp error:", err_out
elif method == "ET":  
    n_est = 30
    n_ft = 30 # 30 or 10 both seem OK     
    min_split = 4 # min
    for n_est in [30]: #[10,20,30,40,50]:
        for n_ft in [10]: # [5,10,20,30,40,50]:
            for min_split in [4]: # [2,3,4,5,6,10,20]:
                regr = ExtraTreesClassifier( n_estimators = n_est, max_features = n_ft, min_samples_split = min_split )
                regr.fit( X_train_enh[tr_set,:], t_train[tr_set] )
                # get in sample and out of sample errors
                Y_in = regr.predict( X_train_enh[tr_set,:] )
                Y_out = regr.predict( X_train_enh[val_set,:] )
                err_in, mat_in = eval_results( t_train[tr_set], Y_in )
                err_out, mat_out = eval_results( t_train[val_set], Y_out )  
                print prefix, "n_est:", regr.n_estimators, "max_ft", regr.max_features, "min_sp_spl", regr.min_samples_split, "in-sp error:", err_in, "out-sp error:", err_out
elif method == "GG":
    flags = [True]
    
    for idx, shr_cov in enumerate(flags):
        regr = GaussianGenerativeModel(isSharedCovariance=shr_cov)
        regr.fit(X_train_enh[tr_set,:], t_train[tr_set])
        # get in sample and out of sample errors
        Y_in = regr.predict( X_train_enh[tr_set,:] )
        Y_out = regr.predict( X_train_enh[val_set,:] )
        err_in, mat_in = eval_results( t_train[tr_set], Y_in )
        err_out, mat_out = eval_results( t_train[val_set], Y_out )  
        np.savetxt(prefix+"shrcov%d_mat_in.csv" % idx, mat_in, delimiter = ',')
        np.savetxt(prefix+"shrcov%d_mat_out.csv" % idx, mat_out, delimiter = ',')
        print prefix, "shared_covariance:", shr_cov, "in-sp error:", err_in, "out-sp error:", err_out
elif method == "LR":   
    eta = 0.0001
    for idx,lamda in enumerate([ 0, 0.0001, 0.001, 0.01, 0.1, 1 ]):
        regr = LogisticRegression(eta=eta, lambda_parameter=lamda)
        regr.fit(X_train_enh[tr_set,:], t_train[tr_set])
        # get in sample and out of sample errors
        Y_in = regr.predict( X_train_enh[tr_set,:] )
        Y_out = regr.predict( X_train_enh[val_set,:] )
        err_in, mat_in = eval_results( t_train[tr_set], Y_in )
        err_out, mat_out = eval_results( t_train[val_set], Y_out )  
        np.savetxt(prefix+"%d_mat_in.csv" % idx, mat_in, delimiter = ',')
        np.savetxt(prefix+"%d_mat_out.csv" % idx, mat_out, delimiter = ',')
        print prefix, "lambda:", lamda, "in-sp error:", err_in, "out-sp error:", err_out
elif method == "SV":
    rg = [0.01, 0.1, 1, 10, 100] +  [0.05, 0.5, 5, 50, 500] 
    rg = [1e-4,5e-4,1e-3,5e-3,1e-2]
    rg = [1e-7, 1e-6,1e-5,1e-4]
    rg = [5e-5,7.5e-5,1e-4,2e-4,3e-4,4e-4,5e-4,6e-4,7e-4,8e-4,9e-4]
    rg = [9e-5,9.5e-5,10e-5]
    rg = [1e-7,1e-6,1e-5]
    rg = [0.01]
    rg = [5.0]
    rg = [0.01, 0.1, 1, 10, 100] +  [0.05, 0.5, 5, 50, 500] 
    rg = [ 1e-4 ]
    rg.sort()
    dgs = [3]
    for dg in dgs:
        for cval in rg: 
            for gm in [ 0.5, 1.0, 2.0, 3.0 ]:
                for cf in [0.0 ]:
                    for sp in [ None ]:
                        regr = SVC(C=cval, kernel = 'poly',degree = dg, gamma = 1.0/102/gm, coef0 = cf )
                        #regr = SVC(C=cval, kernel = 'sigmoid', gamma = gm, coef0 = cf )      
                        #regr = SVC(C=cval, kernel = 'rbf', gamma = gm, coef0 = cf )                        
                        regr.fit(X_train_enh[tr_set,:], t_train[tr_set])
                        # get in sample and out of sample errors
                        Y_in = regr.predict( X_train_enh[tr_set,:] )
                        Y_out = regr.predict( X_train_enh[val_set,:] )
                        err_in, mat_in = eval_results( t_train[tr_set], Y_in )
                        err_out, mat_out = eval_results( t_train[val_set], Y_out )  
                        #np.savetxt(prefix+"_mat_in.csv" , mat_in, delimiter = ',')
                        #np.savetxt(prefix+"_mat_out.csv" , mat_out, delimiter = ',')
                        print prefix, "Degree:", dg, "C:", cval, "gamma:", gm, "coef0:", cf, "in-sp error:", err_in, "out-sp error:", err_out

#np.savetxt(prefix+"_mat_in.csv", mat_in, delimiter = ',')
#np.savetxt(prefix+"_mat_out.csv", mat_out, delimiter = ',')
#print prefix, "n_est:", n_est, "max_ft", n_ft, "min_sp_spl", min_split, "in-sp error:", err_in, "out-sp error:", err_out
#print prefix, "n_est:", regr.n_estimators, "max_ft", regr.max_features, "min_sp_spl", regr.min_samples_split, "in-sp error:", err_in, "out-sp error:", err_out

SV_poly_cont_tr_1hf Degree: 3 C: 0.0001 gamma: 0.5 coef0: 0.0 in-sp error: 0.0654569021387 out-sp error: 0.179520414776
SV_poly_cont_tr_1hf Degree: 3 C: 0.0001 gamma: 1.0 coef0: 0.0 in-sp error: 0.0661049902787 out-sp error: 0.165262475697
SV_poly_cont_tr_1hf Degree: 3 C: 0.0001 gamma: 2.0 coef0: 0.0 in-sp error: 0.143875567077 out-sp error: 0.230071289695
SV_poly_cont_tr_1hf Degree: 3 C: 0.0001 gamma: 3.0 coef0: 0.0 in-sp error: 0.145819831497 out-sp error: 0.230719377835


In [38]:
# saving feature importances from Random Forest
#rfregr = regr
#np.savetxt( "feature_importances.csv", rfregr.feature_importances_, delimiter = "," )

#pct = np.percentile(rfregr.feature_importances_,rm_perc)
#imp_idx = regr.feature_importances_ > pct
#len(imp_idx[imp_idx])


In [157]:
# Use selecte parameters to generate results
from util import write_predictions
method = "RF"
method = "SV"
method = "ET"

if( method == "RF" ):
    regr = RandomForestClassifier(n_estimators = 30, max_features = 30, min_samples_split = 4 )
    regr.fit( X_train, t_train )
    Y_test = regr.predict( X_test )
    # write files for kaggles
    write_predictions(Y_test, test_ids, "rf_ft102_n30_mft30_spl4.csv")  
elif( method == "ET" ):
    feat_imp = np.loadtxt( "feature_importances.csv", delimiter = ","  )
    pct = np.percentile(feat_imp,40)
    imp_idx = feat_imp > pct
    regr = ExtraTreesClassifier(n_estimators = 30, max_features = 10, min_samples_split = 4 )  
 
    # Use 60% of features
    #regr.fit( X_train[:,imp_idx], t_train )
    #Y_test = regr.predict( X_test[:,imp_idx] )
    
    #use full features
    regr.fit( X_train, t_train )
    Y_test = regr.predict( X_test )
    
    Y_in = regr.predict( X_train )
    err_in, mat_in = eval_results( t_train, Y_in )
    print "In-sample error:", err_in
    
    # write files for kaggles
    write_predictions(Y_test, test_ids, "et_ft102_n30_mft10_spl4.csv")   
elif( method == "SV"):
    #regr = SVC(C=0.01, kernel = 'poly',degree = 2, gamma = 1.0/204, coef0 = 0.0 )
    #regr = SVC(C=10.0, kernel = 'rbf', gamma = 1.0/(102*35), coef0 = 0.0 )
    regr = SVC(C=1e-4, kernel = 'poly', degree = 3, gamma = 1.0/(102*1), coef0 = 0.0 )
    regr.fit( X_train, t_train )
    Y_test = regr.predict( X_test )
    # write files for kaggles
    write_predictions(Y_test, test_ids, "svc_poly_C00001_deg3.csv")  

In-sample error: 0.0184705119896


In [ ]:
# look at stats of t_train, to decide grouping of training set and validation set
from collections import Counter
all_perc=[]
for i in [0,1]:
    if i == 0:
        t_set = t_train
    else:    
        t_set = t_train[0:len(t_train)/2]
    ct = Counter(t_set)
    tot = sum(ct.values(),0.0) / 100.0
    for k in ct:
        ct[k] /= tot 
    all_perc.append(ct)
all_perc, all_perc[1] - all_perc[0], all_perc[0] -all_perc[1] # doesn't return negative value

In [ ]:
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

direc = "train"
index = 0
i = -1
X = None
classes = []
ids = [] 
for datafile in os.listdir(direc):
    if datafile == '.DS_Store':
        continue
    i += 1
    # extract id and true class (if available) from filename
    id_str, clazz = datafile.split('.')[:2]
    ids.append(id_str)
    # add target class if this is training data
    try:
        classes.append(util.malware_classes.index(clazz))

    except ValueError:
        # we should only fail to find the label in our list of malware classes
        # if this is test data, which always has an "X" label
        assert clazz == "X"
        classes.append(-1)
    if i == index:
        break

tree = ET.parse(os.path.join(direc,datafile))
tags = []
for el in tree.iter():
    call = el.tag
    tags.append(call)
tags = set(tags)
valid_tags.symmetric_difference(tags)
#el.getchildren()[0].getchildren()[0].getchildren()[0].getchildren()[0].items()

